## Set-up

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from scipy.ndimage.filters import convolve
from scipy.ndimage.filters import gaussian_filter
import numpy as np
import time

In [ ]:
el = 151  # number of elements per side in the microstructure
H = 2  # number of phases in the microstructure
vf = .4  # volume fraction of phase 0
iA = 0  # phase A of correlation
iB = 0  # phase B of correlation

## Generate Microstructure

In [ ]:
base = np.random.random((el, el))

r2a = np.random.randint(2, 20)
r2b = np.random.randint(2, 20)
weights = np.random.random(size=(r2a, r2b))

raw = convolve(base, weights, mode='wrap')

blur = gaussian_filter(raw, sigma=1)

amin = blur.min()
amax = blur.max()
scaled = (blur-amin)/(amax-amin)

micr = scaled > vf

In [ ]:
plt.figure(figsize=[5, 4])
ax = plt.imshow(micr, origin='lower',
                interpolation='none', cmap='gray')
plt.colorbar(ax)
plt.title('microstructure')
plt.show()

## Compute Microstructure-Function

In [ ]:
mf = np.zeros((H, el, el))

for h in xrange(H):
    mf[h, ...] = micr[...] == h
    
frac = np.sum(mf[0, ...])/np.float32(mf[0, ...].size)
    
print "volume fraction phase 0: %s" % np.round(frac, 2)

In [ ]:
plt.figure(figsize=[10, 4])

plt.subplot(121)
ax = plt.imshow(mf[0, ...], origin='lower',
                interpolation='none', cmap='gray')
plt.colorbar(ax)
plt.title('mf[0, :, :]')
plt.subplot(122)
ax = plt.imshow(mf[1, ...], origin='lower',
                interpolation='none', cmap='gray')
plt.colorbar(ax)
plt.title('mf[1, :, :]')
plt.show()

## Calculate 2-pt Spatial Statistics: Naive Approach

In [ ]:
st = time.time()

ff_v1 = np.zeros((el, el), dtype='float32')

S = np.float32(el**2)

set1 = np.float32(mf[iA, ...])
set2 = np.float32(mf[iB, ...])

for ii in xrange(el):
    for jj in xrange(el):
        tmp = np.roll(set2, shift=ii, axis=0)
        tmp = np.roll(tmp, shift=jj, axis=1)
        ff_v1[ii, jj] = np.sum(set1*tmp)/S
        
timeT = np.round(time.time()-st, 5)
print "correlation computed: %s s" % timeT

## Calculate 2-pt Spatial Statistics: FFT Approach

In [ ]:
st = time.time()

M = np.zeros((H, el, el), dtype='complex128')

for h in xrange(H):
    M[h, ...] = np.fft.fftn(mf[h, ...], axes=[0, 1])

S = el**2

M1 = M[iA, ...]
mag1 = np.abs(M1)
ang1 = np.arctan2(M1.imag, M1.real)
exp1 = np.exp(-1j*ang1)
term1 = mag1*exp1

M2 = M[iB, ...]
mag2 = np.abs(M2)
ang2 = np.arctan2(M2.imag, M2.real)
exp2 = np.exp(1j*ang2)
term2 = mag2*exp2

FFtmp = term1*term2/S

ff_v2 = np.fft.ifftn(FFtmp, [el, el], [0, 1]).real    

timeT = np.round(time.time()-st, 5)
print "correlation computed: %s s" % timeT

## Compare Spatial-Statistics

In [ ]:
plt.figure(figsize=[11, 4])

plt.subplot(121)
ff_c = np.fft.fftshift(ff_v1)
ax = plt.imshow(ff_c, origin='lower',
                interpolation='none', cmap='gray')
plt.colorbar(ax)
plt.title('Correlation (Naive method): %s and %s' % (iA, iB))

plt.subplot(122)
ff_c = np.fft.fftshift(ff_v2)
ax = plt.imshow(ff_c, origin='lower',
                interpolation='none', cmap='gray')
plt.colorbar(ax)
plt.title('Correlation (FFT method): %s and %s' % (iA, iB))

plt.show()